# Script 4 - TsTs_T

Runs different models for data which are:

- Target: Difference in Engineered score of next week to this week

- Time series attributes for past weeks: Difference in Observations of each past weeks


In [1]:
import pandas as pd
from sklearn import linear_model
import numpy as np

In [2]:
cols = ['Assets', 'Actions', 'Competency',
       'Form_record', 'Form_template', 'Incident', 'Users']

CentralStatistics = pd.read_csv('Central_Statistics.csv')

for method in ['A', '-26']:
    for datatype in ['TsTs_T']:
        for score in ['2_1', '3_1', '2_2', '3_2']:
            for nweeks in range(2, 13):
                for fsval in [0.2, 0.3, 0.4]:
                    for fold in range(1, 6):


                        # Training

                        df = pd.read_csv(f'./{datatype}_SplitData/{datatype}_{method}_{score}_{nweeks}_Train_{fold}.csv')

                        y = df[['Target']]

                        attrblist = list()
                        for attrb in cols:

                            attrbCol = [col for col in df.columns if attrb in col]
                            x = df[attrbCol]
                            
                            lm = linear_model.LinearRegression()
                            try:
                                
                                model = lm.fit(x,y)
                            except:
                                continue

                            if lm.score(x,y) > fsval:
                                attrblist.extend(attrbCol)
                        
                        if not attrblist:
                            CentralStatisticCol = {'Data': [datatype], 'Scoretype': [score], 'Method': [method], 'NWEEKS': [nweeks],
                                 'r': [lm.score(x,y)], 'fsval': [fsval], 'Fold': [fold], 'TP': [(tp1 + tp2)/2], 'BFP': [(bfp1+bfp2)/2], 
                                'Sens': [(s1+s2)/2], 'TP1': [tp1], 'TP2': [tp2], 'BFP1': [bfp1], 'BFP2': [bfp2], 'Sens1': [s1], 'Sens2': [s2]}
                            
                            CentralStatistics = CentralStatistics.append(pd.DataFrame(CentralStatisticCol))
                            continue
                            
                        x = df[attrblist]

                        lm = linear_model.LinearRegression()
                        
                        try:
                            model = lm.fit(x,y)
                        except:
                            continue

                        # Testing
                        test = pd.read_csv(f'./{datatype}_SplitData/{datatype}_{method}_{score}_{nweeks}_Test_{fold}.csv')
                        out = test[test.columns[0:4]]

                        xTest = test[attrblist]

                        yPred = lm.predict(xTest)

                        out['PredScore'] = yPred 
                        out['ObsScore'] = test['Target']

                        final = pd.DataFrame()

                        for week, data in out.groupby('Week'):
                            data.index = range(len(data))

                            O95 = np.quantile(data['ObsScore'], .95)
                            O05 = np.quantile(data['ObsScore'], .05)

                            P95 = np.quantile(data['PredScore'], .95)
                            P05 = np.quantile(data['PredScore'], .05)

                            data['Obs'] = ['Increase' if data.loc[i]['ObsScore'] > O95 else 'Decrease' if data.loc[i]['ObsScore'] < O05 else 'Normal' for i in range(len(data))]
                            data['Pred'] = ['Increase' if data.loc[i]['PredScore'] > P95 else 'Decrease' if data.loc[i]['PredScore'] < P05 else 'Normal' for i in range(len(data))]

                            final = pd.concat([final, data])

                        n1 = len(final[final['Pred'] == 'Increase'])
                        n2 = len(final[final['Pred'] == 'Decrease'])

                        o1 = len(final[final['Obs'] == 'Increase'])
                        o2 = len(final[final['Obs'] == 'Decrease'])

                        tp1 = len(final[(final['Pred'] == 'Increase') & (final['Obs'] == final['Pred'])])/n1
                        tp2 = len(final[(final['Pred'] == 'Decrease') & (final['Obs'] == final['Pred'])])/n2
                        bfp1 = len(final[(final['Obs'] == 'Decrease') & (final['Pred'] == 'Increase')])/n1
                        bfp2 = len(final[(final['Obs'] == 'Increase') & (final['Pred'] == 'Decrease')])/n2

                        s1 = len(final[(final['Obs'] == 'Increase') & (final['Obs'] == final['Pred'])])/o1
                        s2 = len(final[(final['Obs'] == 'Decrease') & (final['Obs'] == final['Pred'])])/o2


                        CentralStatisticCol = {'Data': [datatype], 'Scoretype': [score], 'Method': [method], 'NWEEKS': [nweeks],
                                 'r': [lm.score(x,y)], 'fsval': [fsval], 'Fold': [fold], 'TP': [(tp1 + tp2)/2], 'BFP': [(bfp1+bfp2)/2], 
                                'Sens': [(s1+s2)/2], 'TP1': [tp1], 'TP2': [tp2], 'BFP1': [bfp1], 'BFP2': [bfp2], 'Sens1': [s1], 'Sens2': [s2]}

                        CentralStatistics = CentralStatistics.append(pd.DataFrame(CentralStatisticCol))

                    CentralStatistics.to_csv('Ts.csv', index = None)
            

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_9621/1650795201.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out['PredScore'] = yPred
/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_9621/1650795201.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  out['ObsScore'] = test['Target']
/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_9621/1650795201.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index